# Handlers in xpytex

In [1]:
from ast import parse, Constant, Name, Attribute, Compare, BoolOp, BinOp, Call
from xpytex import expressions, constants, names, attributes, comparisons, bool_operations, binary_operations, function_calls
from xpytex.utils import displaymath

`xpytex` uses _handlers_. Each one handles a certain type of AST. The "expression" handler mutually depends on all the handlers of different expressions. This means that, for example, the "binary operations" handler (because "binary operations" are a type of expression) depends on the expression handler, this is because a binary expression is defined as `<expr><binary_operator><expr>`, and the only one who knows how to parse expressions is, well... the expressions handler.

Out of the box, the handlers are defined for some expressions. Athough, xpytex is designed to have them be interchangable, that's why we have to `register` them. Each handler has to have a method called `latexify`.

In [2]:
expressions.register_handler(Constant, constants)
expressions.register_handler(Name, names)
expressions.register_handler(Attribute, attributes)
expressions.register_handler(Compare, comparisons)
expressions.register_handler(BoolOp, bool_operations)
expressions.register_handler(BinOp, binary_operations)
expressions.register_handler(Call, function_calls)

In [3]:
code = 'a > b'
AST = parse(code) # This generates a Module AST
AST = AST.body # Gets the lines of the Module
AST = AST[0] # Gets the first line
AST = AST.value # Gets the contents of the line

In [4]:
displaymath( expressions.latexify(AST) )

<IPython.core.display.Math object>

There, we parsed our first expression!
Let's parse another!

In [5]:
code = '1 < 3'
AST = parse(code) # This generates a Module AST
AST = AST.body # Gets the lines of the Module
AST = AST[0] # Gets the first line
AST = AST.value # Gets the contents of the line

```python
displaymath( expressions.latexify(AST) ) # -> Throws KeyError: <class 'int'>
```

This is because the default `constants` handler requires sub-handlers for each data type. All the datatypes that qualify as "Constants" are:
`str`, `bytes`, `bool`, `int`, `float`, `complex` (And also `None` and `Ellipsis`)
Let's add some! Each handler has to be a callable. When called, it should receive a value and return a `str` that is the corresponding LaTeX code. The type of the value has to be a valid constant type. 

In [6]:
# A simple int handler
def int_to_latex(x: int) -> str: return str(x)

# Register the sub-handler to the constants handler
constants.register_handler(int, int_to_latex)

# Let's try again :)
displaymath( expressions.latexify(AST) )

<IPython.core.display.Math object>

Let's try some other handlers!
For example, given that "complex" is another type of constant, let's try to create one for them.

Given that a [complex number](https://en.wikipedia.org/wiki/Complex_number) in Python is represented by the form "bj", let's try to adapt it to print "bi" (As it's more usual when doing math).

In [7]:
z = 2+3j
z_str = str(z)
print(f'Z = '+z_str)

# Let's replace the 'j' for an 'i'
z_str = z_str.replace('j', 'i')
print(f'Z = '+z_str)

# And remove the parentheses...
z_str = z_str.replace('(', '')
z_str = z_str.replace(')', '')
print(f'Z = '+z_str)

Z = (2+3j)
Z = (2+3i)
Z = 2+3i


That looks great! Let's package it all up into a single function.

In [8]:
def complex_to_latex(z: complex) -> str:
    z_str = str(z)
    z_str = z_str.replace('j', 'i')
    z_str = z_str.replace('(', '')
    z_str = z_str.replace(')', '')
    return z_str

displaymath(complex_to_latex(3j))

<IPython.core.display.Math object>

And now, register it as a `constants` handler.

In [9]:
constants.register_handler(complex, complex_to_latex)

# Let's see it in action!
code = '3j > 0j'
AST = parse(code) # This generates a Module AST
AST = AST.body # Gets the lines of the Module
AST = AST[0] # Gets the first line
AST = AST.value # Gets the contents of the line

displaymath( expressions.latexify(AST) )

<IPython.core.display.Math object>

Hmm... "0i" is fine, but what if we wanted to display it only as "0"? Well, for that let's change the handler function

In [10]:
def complex_to_latex(z: complex) -> str:
    if z == 0: 
        return '0'
    else:
        z_str = str(z)
        z_str = z_str.replace('j', 'i')
        z_str = z_str.replace('(', '')
        z_str = z_str.replace(')', '')
        return z_str

# And let's test it!
z = 0j
expected_z = '0'

w = 3j
expected_w = '3i'

print('Test 1')
print(f'{complex_to_latex(z)} = {expected_z} is {complex_to_latex(z) == expected_z}')
print()
print('Test 2')
print(f'{complex_to_latex(w)} = {expected_w} is {complex_to_latex(w) == expected_w}')

Test 1
0 = 0 is True

Test 2
3i = 3i is True


Great! Our function passes our loosely defined tests. Let's register it as a constants handler.

In [11]:
constants.register_handler(complex, complex_to_latex)

# Let's see it in action!
code = '3j > 0j'
AST = parse(code) # This generates a Module AST
AST = AST.body # Gets the lines of the Module
AST = AST[0] # Gets the first line
AST = AST.value # Gets the contents of the line

displaymath( expressions.latexify(AST) )

<IPython.core.display.Math object>